In [ ]:
import pandas as pd
import torch
import LTNtorch as ltn

### USE LTNorch for static dataset


pip install git+https://github.com/tommasocarraro/LTNtorch.git


In [3]:
file_path = 'Stud_E-mobility_data_staticLimit.csv'
data = pd.read_csv(file_path)

In [4]:
# Define a function to apply the given rules to the dataset
def apply_battery_rules(row):
    if row['BATTERY_SOC'] > 80:
        row['ACTION'] = 'Discharge Battery for E-Car Charging'
    elif row['BATTERY_SOC'] > 40:
        if row['GARAGE_EXTERNAL_POWER'] > row['DEMAND_LIMIT']:
            row['ACTION'] = 'Discharge Battery for E-Car Charging'
        else:
            row['ACTION'] = 'No Action'
    elif row['BATTERY_SOC'] >= 15:
        if row['GARAGE_EXTERNAL_POWER'] > row['DEMAND_LIMIT']:
            row['ACTION'] = 'Discharge Battery for E-Car Charging'
        else:
            row['ACTION'] = 'Charge Battery from Grid'
    else:
        row['ACTION'] = 'Stop Discharge - Battery Health'
    return row
new_data = data.apply(apply_battery_rules, axis=1)
new_data[['BATTERY_SOC', 'GARAGE_EXTERNAL_POWER', 'DEMAND_LIMIT', 'ACTION']].head()


,BATTERY_SOC,GARAGE_EXTERNAL_POWER,DEMAND_LIMIT,ACTION
0,40.5,1.244171,50.0,No Action
1,40.5,1.244171,50.0,No Action
2,40.5,1.244171,50.0,No Action
3,40.5,1.244171,50.0,No Action
4,40.5,1.244171,50.0,No Action


In [5]:
new_file_path = 'Modified_E-mobility_data.csv'


In [6]:
new_data.to_csv(new_file_path, index=False)

## Predicates:
- `P_charge(soc, power, limit)`: True if the battery should charge based on SOC, external power, and demand limit.
- `P_discharge(soc, power, limit)`: True if the battery should discharge.
- `P_stop(soc)`: True if the battery discharging should stop.

## Formulas:
- For SOC > 80%, `P_discharge(soc, power, limit)` should always be true.
- For 40% < SOC < 80%, `P_discharge(soc, power, limit)` should be true if `power > limit`.
- For SOC < 40%, different behaviors based on power and limit.
- For SOC < 15%, `P_stop(soc)` should be true.


In [ ]:
data = pd.read_csv('Modified_E-mobility_data.csv')
# Normalize data and convert to tensors
soc = torch.tensor(data['BATTERY_SOC'].values, dtype=torch.float32)
power = torch.tensor(data['GARAGE_EXTERNAL_POWER'].values, dtype=torch.float32)
limit = torch.tensor(data['DEMAND_LIMIT'].values, dtype=torch.float32)
# Define the predicates
P_charge = ltn.Predicate("Charge")
P_discharge = ltn.Predicate("Discharge")
P_stop = ltn.Predicate("Stop")
# LTN Variables
soc_var = ltn.Variable("soc", soc)
power_var = ltn.Variable("power", power)
limit_var = ltn.Variable("limit", limit)
# Define rules using LTN
rule1 = ltn.Forall(soc_var, ltn.Implies(soc_var > 80, P_discharge(soc_var, power_var, limit_var)))
rule2 = ltn.Forall(soc_var, ltn.And(ltn.And(soc_var > 40, soc_var <= 80), ltn.Implies(power_var > limit_var, P_discharge(soc_var, power_var, limit_var))))
rule3 = ltn.Forall(soc_var, ltn.And(soc_var < 40, ltn.And(soc_var >= 15, ltn.Implies(power_var > limit_var, P_discharge(soc_var, power_var, limit_var)))))
rule4 = ltn.Forall(soc_var, ltn.And(soc_var < 40, ltn.And(soc_var >= 15, ltn.Implies(power_var <= limit_var, P_charge(soc_var, power_var, limit_var)))))
rule5 = ltn.Forall(soc_var, ltn.Implies(soc_var < 15, P_stop(soc_var)))
# Combine all rules
all_rules = ltn.And(rule1, rule2, rule3, rule4, rule5)
# Ask LTN to evaluate the truth value of the rules
satisfaction = all_rules.tensor
print(f"Satisfaction of the rules in the dataset: {satisfaction.item()}")
